In [3]:
import PyPDF2
import regex as re
from nltk import word_tokenize 
from nltk.util import ngrams
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from dictionary import *
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import json
import os
import glob
from tqdm import tqdm

In [6]:
def get_text(file):
    text = ''
    with open(file, 'rb') as pdfFileObj:
        pdfReader = PyPDF2.PdfReader(pdfFileObj)
        num_pages = len(pdfReader.pages)
        for i in range(num_pages):
            pageObj = pdfReader.pages[i]
            text += pageObj.extract_text()
        text = text.replace('\n', '')
    return text

def get_string_from_list(list):
    string = ''
    for i in range(len(list)):
        string += list[i]
        if i != len(list)-1:
            string += ' '
    return string

def filter(text):
    token = word_tokenize(text)
    token = [word.lower() for word in token]
    token = [word for word in token if word.isalpha()]
    token = [word for word in token if (word == 'down' or not word in stop_words )]
    # remoev single character
    token = [word for word in token if len(word) > 1]
    # remove th
    token = [word for word in token if word != 'th']
    for i in range(len(token)):
        if token[i] == 'per' and token[i+1] == 'cent':
            token[i] = 'percent'
            token[i+1] = ''
    token = [word for word in token if word != '']
    return token

def get_bigram(token):
    token = [stemmer.stem(word) for word in token]
    bigram = ngrams(token, 2)
    return list(bigram)

def get_trigram(token):
    token = [stemmer.stem(word) for word in token]
    trigram = ngrams(token, 3)
    return list(trigram)

In [7]:
def searchAndCount_single(bigram_list, term, hdlist):
    term = stemmer.stem(term)
    count = 0
    for i in range(len(bigram_list)):
        if term in bigram_list[i]:
            for j in range(len(hdlist)):
                if i-7 < 0:
                    for k in range(i+7):
                        if hdlist[j] in bigram_list[k]:
                            count += 1
                elif i+7 > len(bigram_list):
                    for k in range(i, len(bigram_list)):
                        if hdlist[j] in bigram_list[k]:
                            count += 1
                else:
                    for k in range(i-7, i+7):
                        if hdlist[j] in bigram_list[k]:
                            count += 1
    return count


def searchAndCount_double(bigram_list, term1, term2, hdlist):
    term1 = stemmer.stem(term1)
    term2 = stemmer.stem(term2)

    count = 0
    for i in range(len(bigram_list)):
        if term1 in bigram_list[i] and term2 in bigram_list[i]:
            for j in range(len(hdlist)):
                if i-7 < 0:
                    for k in range(i+7):
                        if hdlist[j] in bigram_list[k]:
                            count += 1
                elif i+7 > len(bigram_list):
                    for k in range(i, len(bigram_list)):
                        if hdlist[j] in bigram_list[k]:
                            count += 1
                else:
                    for k in range(i-7, i+7):
                        if hdlist[j] in bigram_list[k]:
                            count += 1
    return count

In [9]:
os.chdir('/home/uttkarsh/IP/MPC')


# opening each file in each folder
def make_high_low():
    file_names = glob.glob('*.pdf')
    for file_name in file_names:
        print(file_name)
        text = get_text(file_name)
        file_name = file_name[:-4] 
        if os.path.exists(file_name + '_trigram' + '.json'):
            os.remove(file_name + '_trigram' + '.json')
        if os.path.exists(file_name + '_bigram' + '.json'):
            os.remove(file_name + '_bigram' + '.json')
        token = filter(text)
        N = len(token)
        string_list = get_string_from_list(token)
        bigram_list = get_bigram(token)
        trigram_list = get_trigram(token)

        

        # make a dictionary of all above values and dump it in a json file
        if(total_hawk + total_dovish == 0):
            if(os.path.exists(file_name + '.pdf')):
                os.remove(file_name + '.pdf')
            if(os.path.exists(file_name + '.png')):
                os.remove(file_name + '.png')
            print('bye bye bitch')
            continue
        apel_net_sentiment =1 + (total_hawk - total_dovish)/(total_hawk + total_dovish)
        new_net_sentiment = 100*(total_hawk - total_dovish)/(.5*N)
        dict['apel_net_sentiment'] = apel_net_sentiment
        dict['new_net_sentiment'] = new_net_sentiment
        with open(file_name + '.json', 'w') as fp:
            json.dump(dict, fp, indent=4)
            
    return None